In [ ]:
#Importer les modules:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
import colorama
from colorama import Fore
from colorama import Style
import os, sys
import matplotlib
from matplotlib import cm
from matplotlib.patches import Circle, Wedge, Rectangle
from math import log10
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, f1_score, precision_score, recall_score 
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import SMOTENC

import scipy.stats as stats
import shap
import lime
from lime import lime_tabular
import random

import statsmodels.api as sm
from statsmodels.formula.api import ols


import ast
import scipy.stats as stats
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import fbeta_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn.inspection import permutation_importance
import math
from datetime import datetime
from datetime import timedelta, date

from functools import reduce
from sklearn.preprocessing import MinMaxScaler

import pickle
from dill import dumps, loads
import dill

In [ ]:
print(pd.__version__)

In [ ]:
#A. IMPORTER LES DONNÉES:

In [ ]:
cd Desktop\Data Science candidature\Projets\Projet 7\Data\

In [ ]:
ls

In [ ]:
#----------------------------------------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------------------------------------
#B. FEATURE ENGINEERING ET CHOIX DE VARIABLES:

#B.1 Généralités des datasets.
    #a. Le nombre de lignes.

    #On a énormément de variables (colonnes) réparties dans 8 datasets. Le 9ème dataset explique la généralité des 
    #colonnes.

    #Notez qu'on a dès le début deux set bien définis:  
    # -un pour l'entraînement 'application_train.csv' et 
    # -un pour le test 'application_test.csv'.

In [ ]:
df_train = pd.read_csv('application_train.csv')
print(df_train.shape)

In [ ]:
df_test = pd.read_csv('application_test.csv')
print(df_test.shape)

In [ ]:
    #On a ensuite des dataset qui apportent des information complémentaires, et donc des variables potentiellement
    #intéressantes pour l'analyse du risque d'impayés.

In [ ]:
b = pd.read_csv('bureau.csv')
print(b.shape)

In [ ]:
bb = pd.read_csv('bureau_balance.csv')

print(bb.shape)

In [ ]:
prev = pd.read_csv('previous_application.csv')
print(prev.shape)

In [ ]:
posh = pd.read_csv('POS_CASH_balance.csv')
print(posh.shape)

In [ ]:
ins = pd.read_csv('installments_payments.csv')
print(ins.shape)

In [ ]:
cc = pd.read_csv('credit_card_balance.csv')
print(cc.shape)

In [ ]:
    #Notez qu'on n'a pas le même nombre de lignes dans tous les datasets, car en fait dans certains d'entre eux on 
    #split pour chaque client plusieurs lignes, qui correspondent à plusieurs crédits pris ou existants associés au même
    #client.
    
    
    #b. Présence de NaN dans les 8 datasets (en pourcentage).

In [ ]:
(100*(df_train.isna().sum()))/len(df_train)

In [ ]:
(100*(df_test.isna().sum()))/len(df_test)

In [ ]:
(100*(b.isna().sum()))/len(b)

In [ ]:
(100*(bb.isna().sum()))/len(bb)

In [ ]:
(100*(prev.isna().sum()))/len(prev)

In [ ]:
(100*(posh.isna().sum()))/len(posh)

In [ ]:
(100*(ins.isna().sum()))/len(ins)

In [ ]:
(100*(cc.isna().sum()))/len(cc)

In [ ]:
    #Notez aussi que même si pas toutes les variables ont des NaN on en a beaucoup.

    #c.Trouver les colonnes communes entres les 8 datasets.

In [ ]:
list_dataset = [df_train, df_test, b, bb, prev, posh, ins, cc]
col_common = set.intersection(*(set(df.columns) for df in list_dataset))

In [ ]:
col_common

In [ ]:
    #Il faut savoir que les 8 dataset n'ont pas tous de colonnes communes. 

In [ ]:
list_dataset = [df_train, df_test, b, prev, posh, ins, cc]
col_common = set.intersection(*(set(df.columns) for df in list_dataset))

In [ ]:
col_common

In [ ]:
list_dataset = [b,bb]
col_common = set.intersection(*(set(df.columns) for df in list_dataset))

In [ ]:
col_common

In [ ]:
    #En fait il y en a 7 qui ont comme colonne commune 'SK_ID_CURR': df_train, df_test, b, prev, posh, ins, cc; mais le 
    #dataset 'bb' n'a pas cette colonne. Néanmoins il a une colonne commune avec le dataset 'b':'SK_ID_BUREAU'.
    #Il suffira de matcher 'b' et 'bb' par la colonne 'SK_ID_BUREAU' pour ensuite matcher le résultat avec tous les 
    #autres dataset grace à la variable: 'SK_ID_CURR'.
    
    
    #d.Type de variables.

In [ ]:
df_train.dtypes

In [ ]:
df_test.dtypes

In [ ]:
b.dtypes

In [ ]:
bb.dtypes

In [ ]:
prev.dtypes

In [ ]:
posh.dtypes

In [ ]:
ins.dtypes

In [ ]:
cc.dtypes

In [ ]:
    #Parmi les variables, on en a aux valeurs non numériques mais catégorielles, celles de type 'object' -> on a donc 
    #besoin de les rendre numériques avec 'One hot encoder'.

    #Dans chaque database on a des variables intéressantes que nous garderons et d'autre non avant de faire le merge
    #général de tous les datasets. On listera ces variables lors des traitements de chaque dataset.

    #Note: DPD -> Days Past Due.
    
    
    

#B.2 Fonctions de traitement des 8 datasets.

#B.2.1 Fonction 'One hot encoder' pour les colonnes catégorielles avec 'get_dummies'.

In [ ]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


In [ ]:
# B.2.2 Prétraitement des bases application_train.csv et application_test.csv.

#Regardons les caractéristiques générales de ces deux datasets.

    #a. Colonnes communes.
    #En regardant le document: 'HomeCredit_columns_description.csv', on dirait que les 2 dataset ont les mêmes colonnes.
    #Vérifions quand même ceci:

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
a = np. intersect1d(df_train. columns, df_test. columns).tolist()
len(a)

In [ ]:
    #Non, les deux dataset n'ont pas les mêmes colonnes.

In [ ]:
aa=df_train.columns.values.tolist()
len(aa)

In [ ]:
list(set(aa) - set(a))

In [ ]:
df_train['TARGET'].unique()

In [ ]:
    #Le dataset de 'train' a en plus la colonne 'TARGET'. On n'éliminera néanmoins pas cette colonne même si elle n'est 
    #pas présente dans le 'test' set, car elle nous permettra d'identifié le train set du test set, le test étant celui
    #qui aura la variable 'TARGET' non renseignée. En plus, cette variable 'TARGET' est notre cible, car d'après le
    #fichier 'HomeCredit_columns_description', elle identifie les clients avec des difficultés de paiement  (1) des 
    #autres clients (0).NOTEZ que 'TARGET' ne prends que 2 valeurs 'O' ou '1'.
    
    
    #b. Nombre de lignes des deux datasets.

In [ ]:
print("Train samples: {}, test samples: {}".format(len(df_train), len(df_test)))

In [ ]:
    #c.Nombre de colonnes à variables catégorielles:

In [ ]:
h=list(df_train.select_dtypes(['object']).columns)
len(h)

In [ ]:
print(pd.DataFrame({'Variable':['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE'],
                    'Elements':[len(set(df_train.NAME_CONTRACT_TYPE)), len(set(df_train.CODE_GENDER)), len(set(df_train.FLAG_OWN_CAR)),len(set(df_train.FLAG_OWN_REALTY)),len(set(df_train.NAME_TYPE_SUITE)),len(set(df_train.NAME_INCOME_TYPE)),len(set(df_train.NAME_EDUCATION_TYPE)),len(set(df_train.NAME_FAMILY_STATUS)),len(set(df_train.NAME_HOUSING_TYPE)),len(set(df_train.OCCUPATION_TYPE)),len(set(df_train.WEEKDAY_APPR_PROCESS_START)),len(set(df_train.ORGANIZATION_TYPE)),len(set(df_train.FONDKAPREMONT_MODE)),len(set(df_train.HOUSETYPE_MODE)),len(set(df_train.WALLSMATERIAL_MODE)),len(set(df_train.EMERGENCYSTATE_MODE))]}))

In [ ]:
print(pd.DataFrame({'Variable':['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE'],
                    'Elements':[len(set(df_test.NAME_CONTRACT_TYPE)), len(set(df_test.CODE_GENDER)), len(set(df_test.FLAG_OWN_CAR)),len(set(df_test.FLAG_OWN_REALTY)),len(set(df_test.NAME_TYPE_SUITE)),len(set(df_test.NAME_INCOME_TYPE)),len(set(df_test.NAME_EDUCATION_TYPE)),len(set(df_test.NAME_FAMILY_STATUS)),len(set(df_test.NAME_HOUSING_TYPE)),len(set(df_test.OCCUPATION_TYPE)),len(set(df_test.WEEKDAY_APPR_PROCESS_START)),len(set(df_test.ORGANIZATION_TYPE)),len(set(df_test.FONDKAPREMONT_MODE)),len(set(df_test.HOUSETYPE_MODE)),len(set(df_test.WALLSMATERIAL_MODE)),len(set(df_test.EMERGENCYSTATE_MODE))]}))

In [ ]:
df_train['CODE_GENDER'].unique()

In [ ]:
df_test['CODE_GENDER'].unique()

In [ ]:
    #La variable 'CODE_GENDER' dans le 'train' dataset a 4 lignes avec pour valeur 'XNA', on doit les effacer.

    #Par ailleurs, on a trois variables catégorielles binaires: 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY'. On a 
    #déjà vu 'CODE_GENDER', regardons les duex autres ('FLAG_OWN_CAR', 'FLAG_OWN_REALTY').

In [ ]:
df_train['FLAG_OWN_CAR'].unique()

In [ ]:
df_test['FLAG_OWN_CAR'].unique()

In [ ]:
df_train['FLAG_OWN_REALTY'].unique()

In [ ]:
df_test['FLAG_OWN_REALTY'].unique()

In [ ]:
    #On confirme qu'elles sont bien binaires. 
    
    #Note: On n'a que 'CODE_GENDER' et 'NAME_INCOME_TYPE' qui n'ont pas le même nombre de unique values, entre le 
    #'train' et le 'test' sets. Pour 'NAME_INCOME_TYPE', il se peut que l'on n'ai pas une catégorie parmi le 'test' set
    #mais pour le 'CODE_GENDER', la valeur 'XNA' étant associé uniquement à 4 lignes dans le 'train' set on peut la
    #considérer comme une 'erreur', ce qui justifie l'élimination de ces 4 lignes.

    #d. Valeurs aberrantes.

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
#Vérifions qu'on n'a pas de ID communs entre le 'train' et le 'test' sets.

In [ ]:
common_ID=df_train.merge(df_test, on='SK_ID_CURR')
common_ID

In [ ]:
    #On n'en n'a pas.
    
    #En regardant les tableaux précédents 'DAYS_EMPLOYED' a une valeur aberrante. En fait toutes ses valeurs sont 
    #'négatives' (ie. car c'est les jours 'avant la demande de crédit'), sauf une le 'max' qui est trop hors contexte et
    #donc à remplacer par 'nan'.

    #On calculera aussi des variables plus parlantes, décrivant des pourcentages.


    #e. Fonction à associer à ces deux datasets.

In [ ]:
def application_train_test(num_rows = None, nan_as_category = False):
    
    df = pd.read_csv('application_train.csv', nrows= num_rows)
    df_test = pd.read_csv('application_test.csv', nrows= num_rows)
    
    # Élimination de la colonne 'TARGET'. (ON NE LE FAIT PAS).
    #df.drop('TARGET', inplace=True, axis=1)
    
    ##Identification des parties train et test.
    #df['Train_Test']=1
    #df_test['Train_Test']=0
    
    # Fusion (merge) des deux base.
    df = df.append(df_test).reset_index(drop=True)
    # On enlève 4 lignes qui uniquement dans la base 'train' ont 'XNA' dans le 'CODE_GENDER'.
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Rendre numériques les variables catégorielles.
    # Variables catégorielles binaires (ie. ayant uniquement 2 catégories).
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
        
   # On utilise One-Hot encode pour les autres variable catégorielles.
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # Attribution de la valeur NaN pour le 'max' de DAYS_EMPLOYED: 365243 -> nan.
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    
    # Calculs de variables plus parlantes (percentages).
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

    #gc.collect()
    
    return df

In [ ]:
df=application_train_test()
df

In [ ]:
# B.2.3 Prétraitement des database b et bb.

#Regardons les caractéristiques générales de ces datasets.

    #a. Nombre de variables catégorielles.

In [ ]:
h0=list(b.select_dtypes(['object']).columns)
len(h0)

In [ ]:
h0

In [ ]:
h0_1=list(bb.select_dtypes(['object']).columns)
len(h0_1)

In [ ]:
h0_1

In [ ]:
print(pd.DataFrame({'Variable':['CREDIT_ACTIVE','CREDIT_CURRENCY','CREDIT_TYPE'],
                    'Elements':[len(set(b.CREDIT_ACTIVE)), len(set(b.CREDIT_CURRENCY)), len(set(b.CREDIT_TYPE))]}))

In [ ]:
print(pd.DataFrame({'Variable':['STATUS'],
                    'Elements':[len(set(bb.STATUS))]}))

In [ ]:
    #On a respectivement 3 et 1 colonnes catégorielles dans b et bb, soit 4 au total suite au merge.
    
    
    #b. Valeurs abérrantes.

In [ ]:
b.describe()

In [ ]:
bb.describe()

In [ ]:
    #On n'a pas de valeurs aberrantes qui ressortent directement.
    
    
    #c. Aspect général.

In [ ]:
b

In [ ]:
bb

In [ ]:
    #Dans ces 2 bases de données on a plusieurs lignes associées à un même client il faut donc prendre les moyennes, min et
    #max, var, sum... selon les variables. On utilise pour ce faire la fonction aggregate() qui permet d'exécuter une 
    #fonction ou une liste de fonctions le long d'un des axes du DataFrame (par défaut 0 -> lignes).


    #d. Fonction à associer à ces deux datasets.

In [ ]:
def bureau_and_balance(num_rows = None, nan_as_category = True):
    b = pd.read_csv('bureau.csv', nrows = num_rows)
    bb = pd.read_csv('bureau_balance.csv', nrows = num_rows)
    
    # Rendre numériques les variables catégorielles de bureau (b) et bureau_balance (bb).
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    b, b_cat = one_hot_encoder(b, nan_as_category)
    
    # Bureau balance (bb): Aggregations et merge avec bureau.csv (b).
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    b = b.join(bb_agg, how='left', on='SK_ID_BUREAU')
    b.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Variables numériques de bureau (b) et bureau_balance (bb). 
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Features catégorielles de bureau (b) et bureau_balance (bb). 
    cat_aggregations = {}
    for cat in b_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    b_agg = b.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    b_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in b_agg.columns.tolist()])
    
    # Identification des crédits actifs -> on utilise une 'numerical aggregation'.
    active = b[b['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    b_agg = b_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    
    # Identification des crédits clos -> on utilise une 'numerical aggregation'.
    closed = b[b['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    b_agg = b_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg
    
    gc.collect()
    return b_agg


In [ ]:
b_agg=bureau_and_balance()
b_agg

In [ ]:
#B.2.4 Prétraitement de la base previous_applications.csv (prev).

#Regardons les caractéristiques générales de ce dataset.

    #a. Nombre de variables catégorielles.

In [ ]:
h1=list(prev.select_dtypes(['object']).columns)
len(h1)

In [ ]:
h1

In [ ]:
print(pd.DataFrame({'Variable':['NAME_CONTRACT_TYPE','WEEKDAY_APPR_PROCESS_START','FLAG_LAST_APPL_PER_CONTRACT','NAME_CASH_LOAN_PURPOSE','NAME_CONTRACT_STATUS','NAME_PAYMENT_TYPE','CODE_REJECT_REASON','NAME_TYPE_SUITE','NAME_CLIENT_TYPE','NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_PRODUCT_TYPE','CHANNEL_TYPE','NAME_SELLER_INDUSTRY','NAME_YIELD_GROUP','PRODUCT_COMBINATION'],
                    'Elements':[len(set(prev.NAME_CONTRACT_TYPE)), len(set(prev.WEEKDAY_APPR_PROCESS_START)), len(set(prev.FLAG_LAST_APPL_PER_CONTRACT)),len(set(prev.NAME_CASH_LOAN_PURPOSE)),len(set(prev.NAME_CONTRACT_STATUS)),len(set(prev.NAME_PAYMENT_TYPE)),len(set(prev.CODE_REJECT_REASON)),len(set(prev.NAME_TYPE_SUITE)),len(set(prev.NAME_CLIENT_TYPE)),len(set(prev.NAME_GOODS_CATEGORY)),len(set(prev.NAME_PORTFOLIO)),len(set(prev.NAME_PRODUCT_TYPE)),len(set(prev.CHANNEL_TYPE)),len(set(prev.NAME_SELLER_INDUSTRY)),len(set(prev.NAME_YIELD_GROUP)),len(set(prev.PRODUCT_COMBINATION))]}))

In [ ]:
    #On a une seule variable binaire 'FLAG_LAST_APPL_PER_CONTRACT'.
    
    
    #b. Valeurs abérrantes.

In [ ]:
prev.describe()

In [ ]:
    #En regardant les tableaux précédents les variables: 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 
    #'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE' et 'DAYS_TERMINATION' ont une valeur aberrante. En fait toutes leurs 
    #valeurs sont toutes 'négatives' (ie. car c'est les jours 'avant la demande de crédit'), sauf une: leur 'max' qui 
    #est trop hors contexte et donc à remplacer par 'nan'.
    
    
    #c. Aspect général.

In [ ]:
prev

In [ ]:
    #Dans cette base de données un client peut avoir 0, 1, 2 demandes de prêt précédentes ou plus dans le Home Credit
    #(information obtenue du document: 'HomeCredit_columns_description'); il faut donc prendre les moyennes, min et max,
    #var, sum... selon les variables. On utilise pour ce faire la fonction aggregate() qui permet d'exécuter une 
    #fonction ou une liste de fonctions le long d'un des axes du DataFrame (par défaut 0 -> lignes).

    #On calculera aussi des variables plus parlantes, décrivant des pourcentages.


    #d. Fonction à associer à ces deux datasets.

In [ ]:
def previous_applications(num_rows = None, nan_as_category = True):
    
    prev = pd.read_csv('previous_application.csv', nrows = num_rows)
    
    # Rendre numériques les variables catégorielles.
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    
    # Attribution de la valeur NaN pour le 'max' des 5 variables avec valeur aberrante: 365243 -> nan.
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    
    # Création de la variable 'APP_CREDIT_PERC' (pourcentage de crédit reçu) : value ask / value received percentage.
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
    # Variables numériques de previous_applications (prev).
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    
    # Variables catégorielles de previous_applications (prev).
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    
    # Identification des applications approuvées.
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    
    #  Identification des applications refusées.
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg
    
    gc.collect()
    return prev_agg


In [ ]:
prev_agg=previous_applications()
prev_agg

In [ ]:
#B.2.5 Prétraitement de la base POS_CASH_balance.csv (posh).

#Regardons les caractéristiques générales de ce dataset.

    #a. Nombre de variables catégorielles.

In [ ]:
h2=list(posh.select_dtypes(['object']).columns)
len(h2)

In [ ]:
h2

In [ ]:
print(pd.DataFrame({'Variable':['NAME_CONTRACT_STATUS'],
                    'Elements':[len(set(posh.NAME_CONTRACT_STATUS))]}))

In [ ]:
    #On a une seule variable catégorielle.
    
    
    #b. Valeurs aberrantes.

In [ ]:
posh.describe()

In [ ]:
    #On n'a pas de valeurs aberrantes qui ressortent directement.
    
    
    #c. Aspect général.

In [ ]:
posh

In [ ]:
    #Dans cette base de données un client peut avoir 0, 1, 2 demandes de prêt précédentes ou plus dans le Home Credit
    #(information obtenue du document: 'HomeCredit_columns_description'); il faut donc prendre les moyennes, min et max,
    #var, sum... selon les variables. On utilise pour ce faire la fonction aggregate() qui permet d'exécuter une 
    #fonction ou une liste de fonctions le long d'un des axes du DataFrame (par défaut 0 -> lignes).

    #On calculera aussi des variables plus parlantes, décrivant des pourcentages.


    #d. Fonction à associer à ces deux datasets.

In [ ]:
def pos_cash(num_rows = None, nan_as_category = True):
    
    posh = pd.read_csv('POS_CASH_balance.csv', nrows = num_rows)
    
    # Rendre numériques les variables catégorielles.
    posh, cat_cols = one_hot_encoder(posh, nan_as_category= True)
    
    # Variables numériques de pos_cash (posh).
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = posh.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # Nombre de comptes pos_cash.
    pos_agg['POS_COUNT'] = posh.groupby('SK_ID_CURR').size()
   
    gc.collect()
    return pos_agg

In [ ]:
pos_agg=pos_cash()
pos_agg

In [ ]:
#B.2.6 Prétraitement de la base installments_payments.csv (ins).

#Regardons les caractéristiques générales de ce dataset.

    #a. Nombre de variables catégorielles.

In [ ]:
h3=list(ins.select_dtypes(['object']).columns)
len(h3)

In [ ]:
    #Il n'y a pas de variables catégorielles.
    
    
    #b. Valeurs aberrantes.

In [ ]:
ins.describe()

In [ ]:
    #On n'a pas de valeurs aberrantes qui ressortent directement.
    
    
    #c. Aspect général.

In [ ]:
ins

In [ ]:
    #Dans cette base de données un client peut avoir 0, 1, 2 demandes de prêt précédentes ou plus dans le Home Credit
    #(information obtenue du document: 'HomeCredit_columns_description'); il faut donc prendre les moyennes, min et max,
    #var, sum... selon les variables. On utilise pour ce faire la fonction aggregate() qui permet d'exécuter une 
    #fonction ou une liste de fonctions le long d'un des axes du DataFrame (par défaut 0 -> lignes).

    #On calculera aussi des variables plus parlantes, décrivant des pourcentages.


    #d. Fonction à associer à ces deux datasets.

In [ ]:
def installments_payments(num_rows = None, nan_as_category = True):
    
    ins = pd.read_csv('installments_payments.csv', nrows = num_rows)

    # Rendre numériques les variables catégorielles.
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    
    # Pourcentage et différence payés à chaque versement (montant payé et valeur du versement).
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    
    # Days past due and days before due (valeurs non negatives).
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    
    
    # Variables numériques de installments_payments (ins).
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Nombre de installments accounts.
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
   
    gc.collect()
    return ins_agg

In [ ]:
ins_agg=installments_payments()
ins_agg

In [ ]:
#B.2.7 Prétraitement de la base credit_card_balance.csv (cc).

#Regardons les caractéristiques générales de ce dataset.

    #a. Nombre de variables catégorielles.

In [ ]:
h4=list(cc.select_dtypes(['object']).columns)
len(h4)

In [ ]:
h4

In [ ]:
print(pd.DataFrame({'Variable':['NAME_CONTRACT_STATUS'],
                    'Elements':[len(set(cc.NAME_CONTRACT_STATUS))]}))

In [ ]:
    #On a une seule variable catégorielle.
    
    
    #b. Valeurs aberrantes.

In [ ]:
cc.describe()

In [ ]:
    #On n'a pas de valeurs aberrantes qui ressortent directement.
    
    
    #c. Aspect général.

In [ ]:
cc

In [ ]:
    #Dans cette base de données un client peut avoir 0, 1, 2 demandes de prêt précédentes ou plus dans le Home Credit
    #(information obtenue du document: 'HomeCredit_columns_description'); il faut donc prendre les moyennes, min et max,
    #var, sum... selon les variables. On utilise pour ce faire la fonction aggregate() qui permet d'exécuter une 
    #fonction ou une liste de fonctions le long d'un des axes du DataFrame (par défaut 0 -> lignes).

    #On calculera aussi des variables plus parlantes, décrivant des pourcentages.


    #d. Fonction à associer à ces deux datasets.

In [ ]:
def credit_card_balance(num_rows = None, nan_as_category = True):
    
    cc = pd.read_csv('credit_card_balance.csv', nrows = num_rows)

    # Rendre numériques les variabkes catégorielles.
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    
     # Variables numériques de credit_card_balance (cc).
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    
    #Nombre de lignes de carte de crédit. 
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()

    gc.collect()
    return cc_agg

In [ ]:
cc_agg=credit_card_balance()
cc_agg

In [ ]:
#B.3 Fusion des différent datasets traités.

    #Notez que l'on peut faire la fusion sans crainte car on a déjà vérifié que les datasets (autres que b, bb, 
    #df_train et df_test) avaient comme unique variable commune 'SK_ID_CURR'.

In [ ]:
df_1=df.copy()

In [ ]:
df_1 = df_1.join(b_agg, how='left', on='SK_ID_CURR')

In [ ]:
df_1 = df_1.join(prev_agg, how='left', on='SK_ID_CURR')

In [ ]:
df_1 = df_1.join(pos_agg, how='left', on='SK_ID_CURR')

In [ ]:
df_1 = df_1.join(ins_agg, how='left', on='SK_ID_CURR')

In [ ]:
df_1 = df_1.join(cc_agg, how='left', on='SK_ID_CURR')

In [ ]:
df_1

In [ ]:
    #On a:
    
    #Pour les colonnes:
    #-df (et donc df_1) -> 248 colonnes
    #-b_agg -> 116 colonnes
    #-prev_agg -> 249 colonnes
    #-pos_agg -> 18 colonnes
    #-ins_agg -> 26 colonnes
    #-cc_agg -> 141 colonnes

    #Soit la somme de toutes les colonnes -> les 798 colonnes que l'on retrouve bien.
    
    #Pour les lignes, on en a 356251, qui correspondent au nombre de 'SK_ID_CURR'.

In [ ]:
(100*(df_1.isna().sum()))/len(df_1)

In [ ]:
(100*(df_1['TARGET'].isna().sum()))/len(df_1)

In [ ]:
len(df_1.columns[df_1.isna().any()].tolist())

In [ ]:
    #On a énormément de variables avec des NAN (résultant de nos variables initiales avec NAN).
    #La variable 'TARGET' à presque 14% de NaN, mais c'est des NaN qui nous permettent de distinguer le 'train' et le
    #'test' sets. Il faudra conserver les NaN de la colonne 'TARGET'.
    
#B.4 Élimination des NaN.
#Comme ce ne sont pas des données faciles à remplir à titre d'expert ou en établissant des règles de gestion (ie. on ne 
#peut pas les inventer), on éliminera directement les colonnes ayant plus de X% de NAN.
    
#On aurait pu le faire dès le début, mais on décide de le faire maintenant car il était important de faire apparaître,
#lors des traitements des bases de données les "variables plus parlantes à dire d'expert". En fait, si on avait 
#éliminer dès le début toutes les variables ayant plus de 10% de NAN, certaines de ces 'variables parlantes à dire
#d'expert' n'auraient pas été listées.
    
#Voyons combien de colonnes ont plus de 50% de NaN.

In [ ]:
null_percentage = (100*(df_1.isna().sum()))/len(df_1)
col_to_drop1 = null_percentage[null_percentage>50]
len(col_to_drop1)

In [ ]:
#Voyons combien de colonnes ont plus de 50% de NaN.

In [ ]:
col_to_drop2 = null_percentage[null_percentage>60]
len(col_to_drop2)

In [ ]:
#Voyons combien de colonnes ont plus de 10% de NaN.

In [ ]:
col_to_drop3 = null_percentage[null_percentage>10]
len(col_to_drop3)

In [ ]:
#Voyons combien de colonnes ont plus de 5% de NaN.

In [ ]:
col_to_drop4 = null_percentage[null_percentage>4]
len(col_to_drop4)

In [ ]:
#Voyons combien de colonnes ont plus de 1% de NaN.

In [ ]:
col_to_drop5 = null_percentage[null_percentage>1]
len(col_to_drop5)

In [ ]:
#Voyons combien de colonnes ont plus de 0% de NaN.

In [ ]:
col_to_drop6 = null_percentage[null_percentage>0]
len(col_to_drop6)

In [ ]:
#On pourra littéralement éliminer les colonnes qui on plus de 4 pourcent de NaN, pour garder une idée d'avoir une 
#population à 99% propre de NaN. (En fait c'est le même nombre de colonnes à éliminer pour 4% et pour 1%). Et on 
#éliminera le 1% des lignes aui resteront avec des NaN.
    
#On rappelle le fait que l'on ne veut pas enlever la colonne 'TARGET' (vue son importance et pour identifier le 'train' 
#du 'test' set, et surtout étant la variable cible qui identifie les clients avec (1) et sans(0) difficultés de 
#paiement).

In [ ]:
del col_to_drop5['TARGET']

In [ ]:
col_to_drop5=col_to_drop5.keys()

In [ ]:
df_2 = df_1.drop(col_to_drop5, axis=1)

In [ ]:
df_2.shape

In [ ]:
df_2.count()

In [ ]:
#Voyons quelles sont les colonnes qui ont encore des NaN.

In [ ]:
aa=df_2[df_2.columns[df_2.isnull().any()]]
aa.columns

In [ ]:
null_percentage_2 = (100*(df_2.isna().sum()))/len(df_2)
col_to_drop7 = null_percentage_2[null_percentage_2>0]
col_to_drop7

In [ ]:
#Effaçons les NaN qui restent. Sauf ceux de la colonne 'TARGET'.

In [ ]:
del col_to_drop7['TARGET']
col_to_drop7=col_to_drop7.keys()

In [ ]:
df_3=df_2.copy()
df_3=pd.DataFrame(df_3)

In [ ]:
df_3=df_3.dropna(subset=col_to_drop7)

In [ ]:
##Vérification

In [ ]:
bb=df_3[df_3.columns[df_3.isnull().any()]]
bb.columns

In [ ]:
#df_3.isna().sum()

In [ ]:
#null_percentage_3 = (100*(df_3.isna().sum()))/len(df_3)
#col_to_drop8 = null_percentage_3[null_percentage_3>0].keys()
#col_to_drop8

In [ ]:
df_3['TARGET'].count()

In [ ]:
#B.5 Doublons.

In [ ]:
dublons=df_3.duplicated(subset=['SK_ID_CURR']).sum()
dublons

In [ ]:
#Notez qu'il n'y a pas de doublons.

In [ ]:
#B.6 Identification des 'train' et 'test' sets.

In [ ]:
train_df_3=df_3[df_3['TARGET'].notnull()]
train_df_3.shape

In [ ]:
##Vérifions encore une fois qu'on n'a plus de NAN dans le 'train' set.

In [ ]:
cc=train_df_3[train_df_3.columns[train_df_3.isnull().any()]]
cc.columns

In [ ]:
test_df_3=df_3[df_3['TARGET'].isnull()]
test_df_3.shape

In [ ]:
##Vérifions encore une fois qu'on n'a plus de NAN dans le 'test' set (mis appart ceux du 'TARGET').

In [ ]:
dd=test_df_3[test_df_3.columns[test_df_3.isnull().any()]]
dd.columns

In [ ]:
#B.7 Quelques statistiques.

In [ ]:
feats = [f for f in train_df_3.columns if f not in ['SK_ID_CURR','TARGET']]  

In [ ]:
train_stats = train_df_3[feats].describe()
train_stats = train_stats.transpose()
train_stats

In [ ]:
#B.8 Normalisation des données.

#Il est recommandé de normaliser les variables qui ont différentes échelles (cf. statistiques en haut). En fait, 
#même si le modèle peut converger sans normalisation ça rend l'entraînement plus difficile et le résultat dépendant 
#des unités utilisées pour les variables.

#Notez que nous avons générer les statistiques précédentes uniquement sur le "set training", ces statistiques sont 
#également utilisées pour normaliser l'ensemble de données de test, ceci car l'ensemble de données de test doivent 
#être projetées dans la même distribution que celle sur laquelle le training fut fait.

#On a décidé de ne pes normaliser la variable cible ('TARGET'). En fait ceci n'est pas vraiment nécessaire, et en 
#plus il s'agit d'un variable binaire.

#On a parmi nos variables, des variables qui permettent uniquement d'identifier les clients: 'SK_ID_CURR',
#et 'index'; elles on ne les normalise pas.

In [ ]:
def norm(x):
    A=(x[feats] - train_stats['mean']) / train_stats['std']
    A['TARGET']=x['TARGET']
    #A['SK_ID_CURR']=x['SK_ID_CURR']
    
    return A 
train_df_n = norm(train_df_3)
test_df_n = norm(test_df_3)


In [ ]:
train_stats_n = train_df_n.describe()
train_stats_n = train_stats_n.transpose()
train_stats_n

In [ ]:
train_df_n['TARGET'].unique()

In [ ]:
train_df_n['TARGET'].value_counts(normalize=True)*100

In [ ]:
#Notez qu'on a un data set avec 2 classes déséquilibrées: ayant ~91.91% de 'sains' et ~8.9% de défauts.

#Vérifions encore une fois qu'on n'a plus de NAN dans le 'train' et le 'test' set normalisé (mis appart 'TARGET' pour 
#le 'test' set).

In [ ]:
ee=train_df_n[train_df_n.columns[train_df_n.isnull().any()]]
ee.columns

In [ ]:
ff=test_df_n[test_df_n.columns[test_df_n.isnull().any()]]
ff.columns

In [ ]:
#Visiblement on a 2 colonnes avec des 'NAN' suite à la normalisation. Il faut comprendre pourquoi, voyons:

In [ ]:
train_df_3['FLAG_MOBIL'].describe()

In [ ]:
train_df_3['FLAG_MOBIL'].unique()

In [ ]:
train_df_3['NAME_FAMILY_STATUS_Unknown'].describe()

In [ ]:
train_df_3['NAME_FAMILY_STATUS_Unknown'].unique()

In [ ]:
#On voit que pour ces deux variables en fait on a une seule valeur, elles n'apportent donc rien, on peut les éliminer.
#Et ce du 'train' et du 'test' sets.

In [ ]:
train_df_n=train_df_n.drop(['FLAG_MOBIL', 'NAME_FAMILY_STATUS_Unknown'], axis=1) 

In [ ]:
test_df_n=test_df_n.drop(['FLAG_MOBIL', 'NAME_FAMILY_STATUS_Unknown'], axis=1) 

In [ ]:
#Vérifions encore une fois qu'on n'a plus de NAN dans le 'train' et le 'test' set normalisé (mis appart 'TARGET' pour 
#le 'test' set).

In [ ]:
ee1=train_df_n[train_df_n.columns[train_df_n.isnull().any()]]
ee1.columns

In [ ]:
ff1=test_df_n[test_df_n.columns[test_df_n.isnull().any()]]
ff1.columns

In [ ]:
#C'est bon pour ce qui est des NAN et de la Normalisation.

#Rajoutons le 'SK_ID_CURR'.

In [ ]:
train_df_n=train_df_n.join(train_df_3['SK_ID_CURR'])

In [ ]:
train_df_n.shape

In [ ]:
test_df_n=test_df_n.join(test_df_3['SK_ID_CURR'])

In [ ]:
test_df_n.shape

In [ ]:
test_df_n=test_df_n.reset_index(drop=True)

In [ ]:
#B.9 Identification de la variable cible ('TARGET').

In [ ]:
train_cible_n = train_df_n.pop('TARGET')
test_cible_n = test_df_n.pop('TARGET')

In [ ]:
 pd.set_option('display.max_columns', None)

In [ ]:
#ATTENTION: Il ne faut pas que les features (ie. les noms des colonnes aient de caractères JSON).

In [ ]:
train_df_n.columns = train_df_n.columns.str.replace("[,]", "_")
train_df_n.columns = train_df_n.columns.str.replace("[[]]", "_")
train_df_n.columns = train_df_n.columns.str.replace("[:]", "_")
train_df_n.columns = train_df_n.columns.str.replace("[{}]", "_")
train_df_n.columns

In [ ]:
test_df_n.columns = test_df_n.columns.str.replace("[,]", "_")
test_df_n.columns = test_df_n.columns.str.replace("[[]]", "_")
test_df_n.columns = test_df_n.columns.str.replace("[:]", "_")
test_df_n.columns = test_df_n.columns.str.replace("[{}]", "_")
test_df_n.columns

In [ ]:
#Base complète:

In [ ]:
Base_complete=train_df_n
Base_complete=pd.DataFrame(Base_complete)

In [ ]:
Base_complete = Base_complete.append(test_df_n).reset_index(drop=True)

In [ ]:
Base_complete.shape

In [ ]:
#Exportons la base complète en format CSV.

In [ ]:
Base_complete.to_csv(r'C:\Users\'titite\Desktop\Data Science candidature\Projets\Projet 7\Data\Base_complete.csv', index = False)

In [ ]:
#Pour des raison de taille avec Heroku, afin de pouvoir travailler dans la version non payante, on fera les tests sur 
#le cloud avec la moitié de la base test. Ce, afin de réduire la mémoire utilisée sur le serveur de Heroku.

#Exportons donc une petite partie de la base test en format CSV.

In [ ]:
Base_test=test_df_n
Base_test=pd.DataFrame(Base_test)
Base_test.shape

In [ ]:
#On prend donc les 41 premières lignes.

In [ ]:
Base_test1=Base_test.head(41)
Base_test1=pd.DataFrame(Base_test1)
Base_test1.shape

In [ ]:
Base_test1

In [ ]:
Base_test1.to_csv(r'C:\Users\'titite\Desktop\Data Science candidature\Projets\Projet 7\Data\Base_test.csv', index = False)

In [ ]:
#----------------------------------------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------------------------------------
#C. LES MODÈLES:

#Construction et entraînement des modèles.
    #Notez que la fonction ".fit", c'est celle qui est chargée d'entraîner notre modèle et que la fonction ".predict", 
    #c'est celle qui est chargée de prédire la variable cible à partir du dataset test.
    
    #Comme ici le 'test' set n'a pas de 'TARGET' renseignés on utilisera la fonction 'cross_val_score' de 'sklearn',
    #pour valider le modèle (ie. pour calculer un AUC plus acurate). En fait, 'cross_val_score', entraîne et teste un 
    #modèle sur plusieurs plis d'un jeu de données, sans pour autant en retourner un (ie. pas de '.predict' possible 
    #par la suite). Cette méthode de validation croisée permet de mieux comprendre les performances du modèle sur 
    #l'ensemble du jeu de données au lieu de le faire sur un seul fractionnement train/test, et nous aide dans les 
    #situations comme la notre pour tester le modèle.
    #De plus, afin de s'assurer que chaque pli de l'ensemble de données a la même proportion d'observations qui font ou 
    #pas défaut ('TARGET'= 1 et 'TARGET'=0), on utilisera 'StratifiedKFold' de 'sklearn' dans le paramètre 'cv' de 
    #'cross_val_score'.
    
    
    #Par ailleurs, comme on a un dataset avec un 'TARGET' binaire aux classes déséquilibrées, on fera l'analyse de
    #la performance des différents modèles (ie. le calcul de leur AUC), non seulement suite à une 'Cross_Validation'
    #et suite à un '.fit/.predict', mais aussi sur deux approches: une première sans et une deuxième avec 'SMOTE'.
    
    #Le défi de travailler avec des ensembles de données aux classes déséquilibrés est que la plupart des techniques 
    #d'apprentissage automatique ignorent et, à leur tour, ont de mauvaises performances sur la classe minoritaire, 
    #et c'est généralement la performance sur la classe minoritaire qui est la plus importante. Une approche pour 
    #traiter alors les ensembles de données aux classes déséquilibrés consiste à suréchantillonner la classe 
    #minoritaire. L'approche la plus simple consiste à dupliquer des exemples dans la classe minoritaire, qui 
    #n'ajoutent aucune nouvelle information au modèle. De nouveaux exemples peuvent être synthétisés à partir des
    #exemples existants. Il s'agit d'un type d'augmentation de données pour la classe minoritaire qui est appelée 
    #SMOTE' (ie. Synthetic Minority Oversampling Technique).
    #'SMOTE' du packaging 'imblearn' fait donc en quelque sorte un 'réquilibrage' des classes.

In [ ]:
kfold = StratifiedKFold(n_splits=5, random_state=100, shuffle=True)

In [ ]:
#Avec scoring 'roc_auc'.(Dans le cas d’une classification avec des classes deséquilibrées il est suggéré d’utiliser l’AUC).

def cross_validation_2(x, y, model):
    
    start_time_cv = datetime.now()
    result= cross_val_score(model, x, y, cv=kfold, scoring="roc_auc", n_jobs=-1)
    end_time_cv = datetime.now()
    time_cv = end_time_cv - start_time_cv
    time=time_cv.seconds
    
    result_table0 = pd.DataFrame(columns=['classifiers', 'auc'])
    result_table0 = result_table0.append({'classifiers':model.__class__.__name__,
                                         'auc':result.mean(),
                                         'time (s)':time}, ignore_index=True)

    print("Score: %f" % result.mean())
    return result_table0

In [ ]:
#------

#C.1 Choix du meilleur modèle: SANS 'SMOTE' (ie. sans réquilibrage des classes).
#Comparaison des AUC résultant d'une "Cross_Validation" et d'un ".fit/.predict" des différents modèles:


#C.1.0 Train et Test sets obtenus du Train set.

#Maintenant, comme le dataset 'test' n'a pas de cibles renseignées pour entraîner et tester le modèle on va diviser le 
#'train' set en 2, une partie pour le training (avec 80% des données) et une partie pour le test (avec 20% des données).
#Pour que les deux parties soient équilibrées on utilise la fonction 'train_test_split' de 'sklearn', sur les bases 
#'train_df_n' et 'train_cible_n', stratifiée sur 'train_cible_n' ('stratify=train_cible_n').

In [ ]:
train_df_n.shape

In [ ]:
train_cible_n.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df_n, train_cible_n, test_size=0.2, random_state=42, stratify=train_cible_n)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train=X_train.reset_index(drop=True)

In [ ]:
 X_test= X_test.reset_index(drop=True)

In [ ]:
y_train=y_train.reset_index(drop=True)

In [ ]:
y_test=y_test.reset_index(drop=True)

In [ ]:
#C.1.1 Définitions des modèles.

#a. DummyRegressor: 
    #On prend la médiane car elle est statistiquement préférable à la moyenne.
    #Sans optimisation des hyperparamètres (pour le DummyRegressor il n'y a pas vraiment des paramètres à optimiser).

In [ ]:
DM_E= DummyRegressor(strategy='median')

In [ ]:
#a.1 AUC résultant d'une 'Cross_Validation'. 

In [ ]:
DM_E_score_auc = cross_validation_2(X_train, y_train, DM_E)

In [ ]:
DM_E_score_auc

In [ ]:
#a.2  AUC résultant d'un '.fit/.predict'.

#Entraînement du modèle ('.fit').

In [ ]:
start_time_E = datetime.now()

In [ ]:
DM_E.fit(X_train, y_train)

In [ ]:
end_time_E = datetime.now()

In [ ]:
DM_time_E = end_time_E - start_time_E

In [ ]:
#Prédiction ('.predict').

In [ ]:
DM_prediction_E=DM_E.predict(X_test)

In [ ]:
AUC_DM_E=roc_auc_score(y_test,DM_prediction_E)
AUC_DM_E

In [ ]:
#Notez que la valeur du roc_auc est de 0.5 ce qui dit que le modèle est non-informatif.

#------


#b. LGBMClassifier.

#Light GBM utilise un algorithme d'apprentissage basé sur un arbre qui grandit verticalement (ie. par feuille) 
#contrairement à d'autres algorithmes qui font grandir l'arbre horizontalement (ie. par niveau). Il est don plus rapide,
#ce qui le rend très utile pour de grands datasets.

#Les hyperparamètres intéressant à être modifiés sont:
# -'learning_rate': Impact de chaque arbre sur le résultat final. GBM fonctionne en commençant par une estimation 
#                   initiale qui est mise à jour à l'aide de la sortie de chaque arbre. Le paramètre d'apprentissage 
#                   contrôle l'ampleur de ce changement dans les estimations.

# -'num_leaves': Nombre de feuilles dans l'arbre complet.

# -'min_child_samples': Nombre minimum de données nécessaires dans une feuille.

# -'n_estimators': Nombre d'arbres boostés à adapter.


#Pour faire l'optimisation des hyperparamètres on utilise 'GridSearchCV' de 'sklearn'.

#b.1 Obtention des meilleurs hyperparamètre.

In [ ]:
#parameters_lgb = {'learning_rate':[0.2,0.1,0.01,0.05,0.001],'num_leaves':range(10,100,10),'min_child_samples':range(500,1000,100),'n_estimators':range(50,300,50)}

In [ ]:
#lgb= GridSearchCV(estimator = LGBMClassifier(), param_grid = parameters_lgb, cv=5, verbose=False)

In [ ]:
#lgb.fit(X_train, y_train)

In [ ]:
#lgb.best_params_

In [ ]:
#Coefficients obtenus par le "meilleur" modèle (ie. celui avec les "meilleurs hyperparamètres"):

In [ ]:
lgb_best = LGBMClassifier(learning_rate=0.05, num_leaves=70, min_child_samples=800,  n_estimators=250)

In [ ]:
#b.2 AUC résultant d'une 'Cross_Validation'. 

In [ ]:
lgb_score_auc = cross_validation_2(X_train, y_train, lgb_best)
lgb_score_auc

In [ ]:
#Notez que la valeur du roc_auc est de l'ordre de 0.74 ce qui dit que le modèle est assez bon, assez informatif.


#------

#c. XGBClassifier

#Les hyperparamètres intéressant à être modifiés sont:
# -'learning_rate': Impact de chaque arbre sur le résultat final. GBM fonctionne en commençant par une estimation 
#                   initiale qui est mise à jour à l'aide de la sortie de chaque arbre. Le paramètre d'apprentissage 
#                   contrôle l'ampleur de ce changement dans les estimations

# -'subsample': Compris entre 0 et 1 c'est le ratio d'échantillonnage pour le training. Par exemple, le régler sur 0,5 
#               signifie que XGBoost échantillonnerait au hasard la moitié des données.

# -'max_depth': Profondeur maximale d'un arbre. L'augmentation de cette valeur rendra le modèle plus complexe et plus 
#               susceptible de overfit. 0 indique aucune limite de profondeur.

# -'n_estimators': Nombre d'arbres boostés à adapter.


#Pour faire l'optimisation des hyperparamètres on utilise 'GridSearchCV' de 'sklearn'.

#c.1 Obtention des meilleurs hyperparamètres.

In [ ]:
#parameters_xgbc = {learning_rate':[0.2,0.1,0.01,0.05,0.001],'subsample':[1,0.5,0.2,0.1],'max_depth' : range(2,11,1),'n_estimators':range(50,300,50)}

In [ ]:
#xgbc= GridSearchCV(estimator = XGBClassifier(), param_grid = parameters_xgbc, cv=5, verbose=False)

In [ ]:
#xgbc.fit(X_train, y_train)

In [ ]:
#xgbc.best_params_

In [ ]:
#Coefficients obtenus par le "meilleur" modèle (ie. celui avec les "meilleurs hyperparamètres"):

In [ ]:
xgbc_best = XGBClassifier(learning_rate=0.2, max_depth=4, n_jobs=-1, random_state=100,scale_pos_weight=11.387150050352467)

In [ ]:
#c.2 AUC résultant d'une 'Cross_Validation'. 

In [ ]:
scores_xgbc = cross_validation_2(X_train, y_train, xgbc_best)
scores_xgbc

In [ ]:
#C.1.2 Comparaison des modèles. 

#a. AUC résultant d'une 'Cross_Validation'. (sur les meilleurs hyperparamètres pour LGBMClassifier et XGBClassifier).

In [ ]:
df_comp=DM_E_score_auc
df_comp=pd.DataFrame(df_comp)

In [ ]:
df_comp=df_comp.append(lgb_score_auc, ignore_index = True)

In [ ]:
df_comp=df_comp.append(scores_xgbc, ignore_index = True)

In [ ]:
df_comp

In [ ]:
#On voit bien que le LGBMClassifier est le meilleur modèle (ie. celui ayant le meilleur AUC),lorsqu'on utilise la 
#méthode de "cross_validation". Vérifions ceci en utilisant ".fit".


#b. AUC résultant d'un '.fit/.predict'. (sur les meilleurs hyperparamètres pour LGBMClassifier et XGBClassifier).

In [ ]:
classifiers = [ lgb_best,xgbc_best]

result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc','time(s)'])
result_table2 = pd.DataFrame(columns=['classifiers', 'auc','time(s)'])
result_table3 = pd.DataFrame(columns=['classifiers', 'auc','accuracy', 'precision','recall','time(s)'])

for cls in classifiers:
    start_time= datetime.now()
    model = cls.fit(X_train, y_train)
    end_time= datetime.now()
    
    #Il est nécessaire de calculer les 'probabilités' d'occurrence car on a des données non équilibrées (ie. plus de 1 que de 0).    
    yproba = model.predict_proba(X_test)[::,1] 
    
    fpr, tpr, _ = roc_curve(y_test,  yproba)
    auc = roc_auc_score(y_test, yproba)
    time= end_time - start_time
    time2=time.seconds
    
    accuracy=accuracy_score(y_test, model.predict(X_test))
    precision=precision_score(y_test, model.predict(X_test))
    recall=recall_score(y_test, model.predict(X_test))
    
    result_table = result_table.append({'classifiers':cls.__class__.__name__,
                                        'fpr':fpr, 
                                        'tpr':tpr, 
                                        'auc':auc,
                                        'time(s)':time}, ignore_index=True)
    
    result_table2 = result_table2.append({'classifiers':cls.__class__.__name__,
                                         'auc':auc,
                                         'time(s)':time2}, ignore_index=True)
    
    result_table3 = result_table3 .append({'classifiers':cls.__class__.__name__,
                                         'auc':auc,
                                         'accuracy':accuracy,
                                         'precision':precision,
                                         'recall':recall,
                                         'time(s)':time2}, ignore_index=True)


result_table.set_index('classifiers', inplace=True)

In [ ]:
#ATTENTION: 
#Dans les paramètre de la 'roc_curve', 'pos_label' indique la valeur associée à la 'classe positive'. Lorsque 
#'pos_label=None', 'pos_label' vaut 1.

#Ainsi, nous le défaut est représenté par 1, du coup l'état de 'défaut' est notre classe positive.

In [ ]:
result_table2.loc[2] = ['DummyRegressor', AUC_DM_E, DM_time_E.seconds]

In [ ]:
result_table2_2=result_table2.reindex([2,0,1])
result_table2_2.reset_index(drop=True)

In [ ]:
result_table3

In [ ]:
#c. Graphique des courbes ROC_AUC:

In [ ]:
fig = plt.figure(figsize=(10,8))

for i in result_table.index:
    plt.plot(result_table.loc[i]['fpr'], 
             result_table.loc[i]['tpr'], 
             label="{}, AUC={:.3f}".format(i, result_table.loc[i]['auc'])
             )
    
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

plt.show()

In [ ]:
#Notez que le meilleur modèle est le LGBMClassifier, ayant le meilleur AUC (ie. le plus proche de 1), comme la montrent
#le tableau et le graphique. Ceci confirme les résultats obtenus en faisant en utilisant "cross_validation". 


#C.1.3 Probabilités prédites par le meilleur modèle: 'LGBMClassifier'.

#a. Sur le 'train set' (qui est le seul qui la colonne 'TARGET' remplie, et du coup sur lequel on se base).

In [ ]:
lgb_best.fit(X_train, y_train)

In [ ]:
train_cible_predLGB=lgb_best.predict(X_test)

In [ ]:
print(classification_report(y_test, lgb_best.predict(X_test)))

In [ ]:
#ATTENTION:
#Précision-> une métrique qui quantifie le nombre de prédictions 'positives' correctes faites (ie. ici le nombre 
#            de défauts correctement prédits).
#            C'est le rapport des exemples positifs correctement prédits divisé par le nombre total d'exemples positifs 
#            qui ont été prédits:
#            Précision = TruePositives / (TruePositives + FalsePositives)
#            Si 0-> aucune précision,
#            Si 1 -> précision totale ou parfaite -> minimise les faux positifs (ie. ici les faux défauts)
#            'combien d'éléments sont correctement classés dans cette classe'.

#Recall-> une métrique qui quantifie le nombre de prédictions positives correctes faites sur toutes les prédictions 
#         positives qui auraient pu être faites.
#         c'est le rapport des exemples positifs correctement prédits divisé par le nombre total d'exemples positifs 
#         qui pourraient être prédits:
#         Rappel = TruePositives / (TruePositives + FalseNegatives)
#         Si 0 -> aucun rappel, si 1 ->rappel complet ou parfait -> minimise les faux négatifs (ie. ici les faux sains)
#         'combien d'éléments d'une classe on été trouvés sur le nombre total d'élément constituant cette classe'.


#Dans notre cas on a un problème avec le recall, qui doit être dû au caractère déséquilibré de nos classes.

In [ ]:
Proba_train = pd.DataFrame({'SK_ID_CURR': X_test['SK_ID_CURR'],'Defaut':y_test,'Predict_Defaut':lgb_best.predict(X_test),'Proba': lgb_best.predict_proba(X_test)[:,1]})
Proba_train

In [ ]:
#Calculons la matrice de confusion:

In [ ]:
y_true=y_test
y_pred=lgb_best.predict(X_test)

In [ ]:
fig = plt.figure(figsize=(10,7))

cf_matrix=confusion_matrix(y_true, y_pred)

group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
  
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt="", cmap='Blues')

In [ ]:
#b. sur le 'test set':

In [ ]:
test_cible_predLGB=lgb_best.predict(test_df_n)

In [ ]:
Proba_test = pd.DataFrame({'SK_ID_CURR': test_df_n.SK_ID_CURR, 'Proba': lgb_best.predict_proba(test_df_n)[:,1]})
Proba_test

In [ ]:
#------

#C.2 Choix du meilleur modèle: AVEC 'SMOTE' (ie. avec réquilibrage des classes).
#Comparaison des AUC résultant d'une "Cross_Validation" et d'un ".fit/.predict" des différents modèles.

#Commençons par appliquer le 'SMOTE'.

In [ ]:
print("Label 1, Before using SMOTE: {} ".format(sum(train_cible_n==1)))
print("Label 0, Before using SMOTE: {} ".format(sum(train_cible_n==0)))

In [ ]:
sm = SMOTE(random_state=42)

In [ ]:
train_df_req, train_cible_req = sm.fit_resample(train_df_n,train_cible_n)

In [ ]:
#ATTENTION le 'SMOTE' on peut le faire uniquement sur le 'train' set, car on a besoin d'avoir la variable 'TARGET'
#définie. Néanmoins, on appliquera le résultat du '.fit' sur le 'train' set  'smoté' pour faire le '.predict'sur le
#'test' set.

In [ ]:
print("Label 1, After using SMOTE: {}".format(sum(train_cible_req==1)))
print("Label 0, After using SMOTE: {}".format(sum(train_cible_req==0)))

In [ ]:
#C.2.0 Train et Test sets obtenus du Train set.

#Maintenant, comme le dataset 'test' n'a pas de cibles renseignées pour entraîner et tester le modèle on va diviser le 
#'train' set en 2, une partie pour le training (avec 80% des données) et une partie pour le test (avec 20% des données).
#Pour que les deux parties soient équilibrées on utilise la fonction 'train_test_split' de 'sklearn', sur les bases 
#'train_df_n' et 'train_cible_n', stratifiée sur 'train_cible_n' ('stratify=train_cible_n').

In [ ]:
train_df_req.shape

In [ ]:
train_cible_req.shape

In [ ]:
X_train_req, X_test_req, y_train_req, y_test_req = train_test_split(train_df_req, train_cible_req, test_size=0.2, random_state=42, stratify=train_cible_req)

In [ ]:
X_train_req.shape, X_test_req.shape, y_train_req.shape, y_test_req.shape

In [ ]:
X_train_req=X_train_req.reset_index(drop=True)

In [ ]:
X_test_req=X_test_req.reset_index(drop=True)

In [ ]:
y_train_req=y_train_req.reset_index(drop=True)

In [ ]:
y_test_req=y_test_req.reset_index(drop=True)

In [ ]:
#Sauvegardons ces bases.

In [ ]:
X_train_req.to_csv(r'C:\Users\'titite\Desktop\Data Science candidature\Projets\Projet 7\Data\X_train_req_saved.csv', index = False)

In [ ]:
X_test_req.to_csv(r'C:\Users\'titite\Desktop\Data Science candidature\Projets\Projet 7\Data\X_test_req_saved.csv', index = False)

In [ ]:
y_train_req.to_csv(r'C:\Users\'titite\Desktop\Data Science candidature\Projets\Projet 7\Data\y_train_req_saved.csv', index = False)

In [ ]:
y_test_req.to_csv(r'C:\Users\'titite\Desktop\Data Science candidature\Projets\Projet 7\Data\y_test_req_saved.csv', index = False)

In [ ]:
#C.2.1 Définitions des modèles.

#a. DummyRegressor: 
    #On prend la médiane car elle est statistiquement préférable à la moyenne.
    #Sans optimisation des hyperparamètres (pour le DummyRegressor il n'y a pas vraiment des paramètres à optimiser).

In [ ]:
DM_E= DummyRegressor(strategy='median')

In [ ]:
#a.1 AUC résultant d'une 'Cross_Validation'. 

In [ ]:
DM_E_score_auc_req = cross_validation_2(X_train_req, y_train_req, DM_E)

In [ ]:
DM_E_score_auc_req

In [ ]:
#a.2  AUC résultant d'un '.fit/.predict'.

#Entraînement du modèle ('.fit').

In [ ]:
start_time_E_req = datetime.now()

In [ ]:
DM_E.fit(X_train_req, y_train_req)

In [ ]:
end_time_E_req = datetime.now()

In [ ]:
DM_time_E_req = end_time_E_req - start_time_E_req

In [ ]:
#Prédiction ('.predict').

In [ ]:
DM_prediction_E_req=DM_E.predict(X_test_req)

In [ ]:
AUC_DM_E_req=roc_auc_score(y_test_req,DM_prediction_E_req)
AUC_DM_E_req

In [ ]:
#Notez que la valeur du roc_auc est de 0.5 ce qui dit que le modèle est non-informatif.

#------


#b. LGBMClassifier.

#Light GBM utilise un algorithme d'apprentissage basé sur un arbre qui grandit verticalement (ie. par feuille) 
#contrairement à d'autres algorithmes qui font grandir l'arbre horizontalement (ie. par niveau). Il est don plus rapide, ce
#qui le rend très utile pour de grands datasets.

#Les hyperparamètres intéressant à être modifiés sont:
# -'learning_rate': Impact de chaque arbre sur le résultat final. GBM fonctionne en commençant par une estimation 
#                   initiale qui est mise à jour à l'aide de la sortie de chaque arbre. Le paramètre d'apprentissage 
#                   contrôle l'ampleur de ce changement dans les estimations

# -'num_leaves': Nombre de feuilles dans l'arbre complet.

# -'min_child_samples': Nombre minimum de données nécessaires dans une feuille.

# -'n_estimators': Nombre d'arbres boostés à adapter.


#Pour faire l'optimisation des hyperparamètres on utilise 'GridSearchCV' de 'sklearn'.

#b.1 Obtention des meilleurs hyperparamètre.

In [ ]:
#parameters_lgb = {'learning_rate':[0.2,0.1,0.01,0.05,0.001],'num_leaves':range(10,100,10),'min_child_samples':range(500,1000,100),'n_estimators':range(50,300,50)}

In [ ]:
#lgb= GridSearchCV(estimator = LGBMClassifier(), param_grid = parameters_lgb, cv=5, verbose=False)

In [ ]:
#lgb.fit(X_train_req, y_train_req)

In [ ]:
#lgb.best_params_

In [ ]:
#Coefficients obtenus par le "meilleur" modèle (ie. celui avec les "meilleurs hyperparamètres"):

In [ ]:
lgb_best = LGBMClassifier(learning_rate=0.05, num_leaves=70, min_child_samples=800,  n_estimators=250)

In [ ]:
#b.2 AUC résultant d'une 'Cross_Validation'. 

In [ ]:
lgb_score_auc_req = cross_validation_2(X_train_req, y_train_req, lgb_best)
lgb_score_auc_req

In [ ]:
#Notez que la valeur du roc_auc est de l'ordre de 0.74 ce qui dit que le modèle est assez bon, assez informatif.


#------

#c. XGBClassifier.

#Les hyperparamètres intéressant à être modifiés sont:
# -'learning_rate': Impact de chaque arbre sur le résultat final. GBM fonctionne en commençant par une estimation 
#                   initiale qui est mise à jour à l'aide de la sortie de chaque arbre. Le paramètre d'apprentissage 
#                   contrôle l'ampleur de ce changement dans les estimations.

# -'subsample': Compris entre 0 et 1 c'est le ratio d'échantillonnage pour le training. Par exemple, le régler sur 0,5 
#               signifie que XGBoost échantillonnerait au hasard la moitié des données.

# -'max_depth': Profondeur maximale d'un arbre. L'augmentation de cette valeur rendra le modèle plus complexe et plus 
#               susceptible de overfit. 0 indique aucune limite de profondeur.

# -'n_estimators': Nombre d'arbres boostés à adapter.


#Pour faire l'optimisation des hyperparamètres on utilise 'GridSearchCV' de 'sklearn'.

#c.1 Obtention des meilleurs hyperparamètres.

In [ ]:
#parameters_xgbc = {learning_rate':[0.2,0.1,0.01,0.05,0.001],'subsample':[1,0.5,0.2,0.1],'max_depth' : range(2,11,1),'n_estimators':range(50,300,50)}

In [ ]:
#xgbc= GridSearchCV(estimator = XGBClassifier(), param_grid = parameters_xgbc, cv=5, verbose=False)

In [ ]:
#xgbc.fit(X_train_req, y_train_req)

In [ ]:
#xgbc.best_params_

In [ ]:
#Coefficients obtenus par le "meilleur" modèle (ie. celui avec les "meilleurs hyperparamètres"):

In [ ]:
xgbc_best = XGBClassifier(learning_rate=0.2, max_depth=4, n_jobs=-1, random_state=100,scale_pos_weight=11.387150050352467)

In [ ]:
#c.2 AUC résultant d'une 'Cross_Validation'. 

In [ ]:
scores_xgbc_req = cross_validation_2(X_train_req, y_train_req, xgbc_best)
scores_xgbc_req

In [ ]:
#C.1.2 Comparaison des modèles. 

#a. AUC résultant d'une 'Cross_Validation'. (sur les meilleurs hyperparamètres pour LGBMClassifier et XGBClassifier).

In [ ]:
df_comp_req=DM_E_score_auc_req
df_comp_req=pd.DataFrame(df_comp_req)

In [ ]:
df_comp_req=df_comp_req.append(lgb_score_auc_req, ignore_index = True)

In [ ]:
df_comp_req=df_comp_req.append(scores_xgbc_req, ignore_index = True)

In [ ]:
df_comp_req

In [ ]:
#On voit bien que le LGBMClassifier est le meilleur modèle (ie. celui ayant le meilleur AUC), lorsqu'on utilise la 
#méthode de "cross_validation". Vérifion ceci en utilisant ".fit".


#b. AUC résultant d'un '.fit/.predict'. (sur les meilleurs hyperparamètres pour LGBMClassifier et XGBClassifier).

In [ ]:
classifiers = [lgb_best, xgbc_best]

result_table_req = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc','time(s)'])
result_table2_req = pd.DataFrame(columns=['classifiers', 'auc','time(s)'])
result_table3_req = pd.DataFrame(columns=['classifiers', 'auc','accuracy', 'precision','recall','time(s)'])

for cls in classifiers:
    start_time_req= datetime.now()
    model = cls.fit(X_train_req, y_train_req)
    end_time_req= datetime.now()
    
    #Il est nécessaire de calculer les 'probabilités' d'occurrence car on a des données non équilibrées (ie. plus de 1 que de 0).    
    yproba = model.predict_proba(X_test_req)[::,1] 
    
    fpr, tpr, _ = roc_curve(y_test_req,  yproba)
    auc = roc_auc_score(y_test_req, yproba)
    time_req= end_time_req - start_time_req
    time2_req=time_req.seconds
    
    accuracy=accuracy_score(y_test_req, model.predict(X_test_req))
    precision=precision_score(y_test_req, model.predict(X_test_req))
    recall=recall_score(y_test_req, model.predict(X_test_req))
    
    result_table_req = result_table_req.append({'classifiers':cls.__class__.__name__,
                                        'fpr':fpr, 
                                        'tpr':tpr, 
                                        'auc':auc,
                                        'time(s)':time_req}, ignore_index=True)
    
    result_table2_req = result_table2_req.append({'classifiers':cls.__class__.__name__,
                                         'auc':auc,
                                         'time(s)':time2_req}, ignore_index=True)
    
    result_table3_req = result_table3_req.append({'classifiers':cls.__class__.__name__,
                                         'auc':auc,
                                         'accuracy':accuracy,
                                         'precision':precision,
                                         'recall':recall,
                                         'time(s)':time2_req}, ignore_index=True)


result_table_req.set_index('classifiers', inplace=True)

In [ ]:
#ATTENTION: 
#Dans les paramètre de la 'roc_curve', 'pos_label' indique la valeur associée à la 'classe positive'. Lorsque 
#'pos_label=None', 'pos_label' vaut 1.

#Ainsi, nous le défaut est représenté par 1, du coup l'état de 'défaut' est notre classe positive.

In [ ]:
result_table2_req.loc[2] = ['DummyRegressor', AUC_DM_E_req, DM_time_E_req.seconds]

In [ ]:
result_table2_req_2=result_table2_req.reindex([2,0,1])
result_table2_req_2.reset_index(drop=True)

In [ ]:
result_table3_req

In [ ]:
#c. Graphique des courbes ROC_AUC:

In [ ]:
fig = plt.figure(figsize=(10,8))

for i in result_table_req.index:
    plt.plot(result_table_req.loc[i]['fpr'], 
             result_table_req.loc[i]['tpr'], 
             label="{}, AUC={:.3f}".format(i, result_table_req.loc[i]['auc'])
             )
    
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

plt.show()

In [ ]:
#Notez que le meilleur modèle est le LGBMClassifier, ayant le meilleur AUC (ie. le plus proche de 1), comme la montrent
#le tableau et le graphique. Ceci confirme les résultats obtenus en faisant en utilisant "cross_validation". 


#C.1.3 Probabilités prédites par le meilleur modèle: 'LGBMClassifier'.

#a. Sur le 'train set' (qui est le seul qui la colonne 'TARGET' remplie, et du coup sur lequel on se base).

In [ ]:
lgb_best.fit(X_train_req, y_train_req)

In [ ]:
train_cible_predLGB=lgb_best.predict(X_test_req)

In [ ]:
print(classification_report(y_test_req, lgb_best.predict(X_test_req)))

In [ ]:
#ATTENTION:
#Précision-> une métrique qui quantifie le nombre de prédictions 'positives' correctes faites (ie. ici le nombre 
#            de défauts correctement prédits).
#            C'est le rapport des exemples positifs correctement prédits divisé par le nombre total d'exemples positifs 
#            qui ont été prédits:
#            Précision = TruePositives / (TruePositives + FalsePositives)
#            Si 0-> aucune précision,
#            Si 1 -> précision totale ou parfaite -> minimise les faux positifs (ie. ici les faux défauts)
#            'combien d'éléments sont correctement classés dans cette classe'

#Recall-> une métrique qui quantifie le nombre de prédictions positives correctes faites sur toutes les prédictions 
#         positives qui auraient pu être faites.
#         c'est le rapport des exemples positifs correctement prédits divisé par le nombre total d'exemples positifs 
#         qui pourraient être prédits:
#         Rappel = TruePositives / (TruePositives + FalseNegatives)
#         Si 0 -> aucun rappel, si 1 ->rappel complet ou parfait -> minimise les faux négatifs (ie. ici les faux sains)
#         'combien d'éléments d'une classe on été trouvés sur le nombre total d'élément constituant cette classe'


#En réquilibrant les classes avec 'SMOTE' le problème du 'Recall' est résolu.

#Sauvegardons donc ce modèle:
#NOTE: Pour el sauvegarder une fois le meilleur modèle choisit et entraîné, on utilise le packaging 'Pickle'.

In [ ]:
#Sauvegardons le modèle entraîné comme un pickle string.

#create an iterator object with write permission - model.pkl
with open('model_pkl', 'wb') as files:
    pickle.dump(lgb_best, files)

In [ ]:
#Explorons un peu les résultats.

In [ ]:
Proba_train_req = pd.DataFrame({'SK_ID_CURR': X_test_req['SK_ID_CURR'],'Defaut':y_test_req,'Predict_Defaut (.predict)':lgb_best.predict(X_test_req),'Proba': lgb_best.predict_proba(X_test_req)[:,1]})
Proba_train_req

In [ ]:
#Calculons la matrice de confusion:
#Obtenue en faisant le '.predict' et donc en donnant le même poids au Faux Positifs et aux Faux Négatifs.

In [ ]:
y_true_req=y_test_req
y_pred_req=lgb_best.predict(X_test_req)

In [ ]:
fig = plt.figure(figsize=(10,7))

cf_matrix=confusion_matrix(y_true_req, y_pred_req)

group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
  
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt="", cmap='Blues')

In [ ]:
#b. sur le 'test set':

In [ ]:
test_cible_predLGB=lgb_best.predict(test_df_n)

In [ ]:
Proba_test_req = pd.DataFrame({'SK_ID_CURR': test_df_n.SK_ID_CURR, 'Proba': lgb_best.predict_proba(test_df_n)[:,1]})
Proba_test_req

In [ ]:
#C.3 Fonction de coût métier.

#Note: On rappelle que Positif=défaut et négatif=sain.

#Dans le domaine du crédit, les faux négatifs (ie. ici les faux sains) sont plus coûteux, on veut donc les diminuer, ou
#autrement dit, leur donner moins de poids.

#On décide d'utiliser la fonction 'fbeta_score' de 'Sklearn', car elle permet de donner moins de poids au 
#'Faux négatifs' (ie. contreparties faussement notées comme saines, c'est à dire contreparties qui en vrai son en 
#défaut mais sont notées comme saines) qui sont moins coûteux que les 'Faux Positifs' (ie les contreparties faussement 
#notées en défaut, c'est à dire contreparties qui en vrai sont saines mais sont notées en défaut).

#Un score F-bêta > 0.85 est considéré comme un excellent score, un score fbeta > 0,7 comme un bon et tout autre score 
#comme un mauvais. 

#Quant au coefficient 'β', il se définit comme:

#                                 β = Cost of False Negative/Cost of False Positive.


#le choix de β, qui est donc à définir par l'expertise métier, déterminera le poids que l'on souhaite donner à chacune 
#des deux mesures:
# -β<1 -> privilégie la précision (si β=0 , alors F_β=Précision)-> minimise les faux positifs (ie. ici les faux défauts)
# -β>1 -> privilégie le recall -> minimise les faux négatifs (ie. ici les faux sains).

#Nous on préfère avoir plus de faux positifs (ie. clients mis en défauts alors qu'ils ne le sont pas) que de faux 
#négatifs (ie. clients non mis en défaut qui devraient l'être). Si l'on veut minimiser les faux négatifs, on
#sélectionnera un β supérieur à 1 (ie. un fort rappel implique un faible taux de faux négatifs).

#Ainsi si le Coût d'un Faux Négatif (un défaut classifié comme sain) est de 30% de la valeur du crédit, et que le coût
#d'un Faux Positif (un sain qui se voit refuser le crédit car classifié négatif) et de 10% (en fait c'est 3 fois moins 
#coûteux qu'un défaut qui obtient le crédit), on peut prendre β=3. Il reste donc à trouver le meilleur threshold (seuil)
#associé à ce β (ie. celui qui donne le meilleur fbeta score, le maximum fbeta score).

#Voyons quel est le couplet (β, threshold) qui donne le meilleur 'fbeta_score':

In [ ]:
y_test_probs=lgb_best.predict_proba(X_test_req)[:,1] #On prend les résultats obtenus de la partie 'test' du training set.

f0_5_scores = []
f1_scores = []
f1_5_scores = []
f2_scores = []
f2_5_scores = []
f3_scores = []
f4_scores = []

precision_scores = []
recall_scores = []

# Définition des seuils (ie. threshold) de probabilité à utiliser, entre 0 et 1.
probability_thresholds = np.linspace(0, 1, num=100)

# Trouver des scores pour chaque seuil (ie. threshold).
for p in probability_thresholds:
    
    y_test_preds = []
    
    for prob in y_test_probs:
        if prob > p:
            y_test_preds.append(1)
        else:
            y_test_preds.append(0)
            
    
    f0_5 = fbeta_score(y_test_req, y_test_preds, beta=0.5)
    f1 = f1_score(y_test_req, y_test_preds)
    f1_5 = fbeta_score(y_test_req, y_test_preds, beta=1.5)
    f2 = fbeta_score(y_test_req, y_test_preds, beta=2)
    f2_5 = fbeta_score(y_test_req, y_test_preds, beta=2.5)
    f3 = fbeta_score(y_test_req, y_test_preds, beta=3)
    f4 = fbeta_score(y_test_req, y_test_preds, beta=4)
    
    prec = precision_score(y_test_req, y_test_preds)
    rec = recall_score(y_test_req, y_test_preds)
        
    f0_5_scores.append(f0_5)
    f1_scores.append(f1)
    f1_5_scores.append(f1_5)
    f2_scores.append(f2)
    f2_5_scores.append(f2_5)
    f3_scores.append(f3)
    f4_scores.append(f4)
    precision_scores.append(prec)
    recall_scores.append(rec)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(probability_thresholds, f0_5_scores , label='f0.5')
ax.plot(probability_thresholds, f1_scores, label='f1')
ax.plot(probability_thresholds, f1_5_scores, label='f1.5')
ax.plot(probability_thresholds, f2_scores, label='f2')
ax.plot(probability_thresholds, f2_5_scores, label='f2.5')
ax.plot(probability_thresholds, f3_scores, label='f3')
ax.plot(probability_thresholds, f4_scores, label='f4')
ax.plot(probability_thresholds, precision_scores, label='Precision')
ax.plot(probability_thresholds, recall_scores, label='Recall')
ax.set_xlabel('Probability Threshold')
ax.set_ylabel('f-beta score')
ax.legend(loc='center right');
ax.set_ylim(0.47, 1.05)

In [ ]:
#On voit bien que les valeurs maximales du fbeta score s’obtiennent pour les β plus petits, et donc quand on favorise 
#la précision (ie. on diminue les Faux Positifs,et donc les Faux Defauts). L'expertise métier ne voulant prioritariser 
#le recall, si l'on se centre sur le β=3 (élue suite aux explications précédentes des rapports de coûts entre faux 
#négatifs et faux positifs) on a:

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

ax.plot(probability_thresholds, f3_scores, label='f3')
ax.plot(probability_thresholds, precision_scores, label='Precision')
ax.plot(probability_thresholds, recall_scores, label='Recall')

ymax = max(f3_scores)
xpos = f3_scores.index(ymax)
xmax = probability_thresholds[xpos]
text= "Local max: x={:.3f}, y={:.3f}".format(xmax, ymax)

ax.set_xlabel('Probability Threshold')
ax.set_ylabel('f-beta score')
ax.legend(loc='center right')
ax.set_ylim(0.47, 1.05)

ax.annotate(text, xy=(xmax, ymax),xytext=(xmax, ymax-0.05),
            arrowprops=dict(facecolor='black', shrink=0.05))

In [ ]:
#Le meilleur seuil (ie. threshold) pour β=3 (lui choisi par expertise métier) est 0.091.
#Ainsi tout ce qui est <=0.91 sera mis comme 'sain', et tout ce qui est >0.091 en défaut.

#Recalculons alors les Défauts prédits en considérant le 'Coût Métier' (ie. le 'Predict_Defaut_Avec_CM').

In [ ]:
Proba_train_req2=Proba_train_req
Proba_train_req2=pd.DataFrame(Proba_train_req2)

In [ ]:
defaut_CM = []

for row in Proba_train_req2['Proba']:
    if row > 0.091 :defaut_CM.append(1)
    else:defaut_CM.append(0)

In [ ]:
Proba_train_req2['Predict_Defaut_Avec_CM']=defaut_CM
Proba_train_req2

In [ ]:
#Recalculons le fbeta score obtenu ainsi:

In [ ]:
y_true_req=y_test_req
y_pred_req_avec_CM=Proba_train_req2['Predict_Defaut_Avec_CM']

In [ ]:
p = precision_score(y_true_req, y_pred_req_avec_CM)
r = recall_score(y_true_req, y_pred_req_avec_CM)
f = fbeta_score(y_true_req, y_pred_req_avec_CM, beta=3)

print('Result: p=%.3f, r=%.3f, f=%.3f' % (p, r, f))

In [ ]:
#On retrouve bien le fbeta score maximum obtenu pour β=3 et on voit bien qu'on donne ainsi plus de poids au recall
#qu'à la précision, diminuant ainsi les 'Faux Négatifs' comme le demande le métier.

#Appliquons maintenant cette nouvelles règle sur le 'test set':

In [ ]:
Proba_test_req2=Proba_test_req
Proba_test_req2=pd.DataFrame(Proba_test_req2)

In [ ]:
defaut_CM_test_set = []

for row in Proba_test_req2['Proba']:
    if row > 0.091 :defaut_CM_test_set.append(1)
    else:defaut_CM_test_set.append(0)

In [ ]:
Proba_test_req2['Predict_Defaut_Avec_CM']=defaut_CM_test_set
Proba_test_req2

In [ ]:
#C.4 Features Importance.

#L'importance locale ->se concentre sur la contribution des caractéristiques pour une prédiction spécifique
#                    ->pertinente lorsque nous voulons expliquer pourquoi une personne spécifique s'est vu refuser un 
#                      prêt à partir d'un modèle.

#L'importance globale -> prend en compte toutes les prédictions.
#                     -> On utilise la fonction '.feature_importances_' associée au 'RandomForestRegressor' de 
#                       'sklearn.ensemble'.


#C.4.1 Features Importance Globale (dépendante de l'algorithme ->.feature_importances_):

In [ ]:
Globla_FI=pd.DataFrame(lgb_best.feature_importances_, X_train_req.columns, columns = ['Importance'])

#On sélectionne les 40 meilleures features avec le [:20].
A=Globla_FI.sort_values(by=['Importance'],ascending=False)[:20] 
A

In [ ]:
#Adaptation pour le code Flask.

In [ ]:
Globla_FI2 = pd.Series(lgb_best.feature_importances_,index=X_train_req.columns).sort_values(ascending=False, axis=0)[:10]

In [ ]:
Globla_FI2

In [ ]:
#Sauvegardons le "explainer" de "LIME" comme un pickle string.
#Cette fois-ci en utilisant "dill", car "pickel" ne fonctionne pas avec "LIME"

#create an iterator object with write permission - expl.pkl

In [ ]:
with open('global_pkl', 'wb') as files:
    dill.dump(Globla_FI2, files)

In [ ]:
#Vérification.

In [ ]:
#with open('global_pkl', 'rb') as files:
#    glob= dill.load(files)

In [ ]:
#B=glob.sort_values(by=['Importance'],ascending=False)[:20] 
#B

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
A.plot(kind='bar', ax=ax,legend=False)
ax.set_xlabel('20 Best Features', size=15)
ax.set_ylabel('Importance', size=15)
ax.tick_params(axis='x', labelrotation =90)

In [ ]:
#Notez que c'est la variable 'PAYMENT_RATE' ='AMT_ANNUITY'/'AMT_CREDIT' (ie. Paiement annuel d'une partie du capital 
#emprunté et des intérêts/ crédit total emprunté), que l'on a crée qui est la plus importante. Lui suivant le 
#'EXT_SOURCE_2' qui est un score provenant d'une external data source (comme un benchmark) et les 'DAYS_BIRTH'.
#Pour ce qui est des 'DAYS_BIRTH' voyons quel est le lien:

In [ ]:
Proba_train_req3=Proba_train_req2
Proba_train_req3=pd.DataFrame(Proba_train_req3)

In [ ]:
Proba_train_req3['DAYS_BIRTH']=X_test_req['DAYS_BIRTH']
Proba_train_req3

In [ ]:
B=round(abs(train_df_3['DAYS_BIRTH'])/365)
B=pd.DataFrame(B)
B['SK_ID_CURR']=train_df_3['SK_ID_CURR']
B=B.rename({'DAYS_BIRTH': 'DAYS_BIRTH_2'}, axis=1)
B

In [ ]:
D=Proba_train_req3.merge(B, on='SK_ID_CURR')
D

In [ ]:
C=D.loc[:,['Predict_Defaut_Avec_CM', 'DAYS_BIRTH_2']]
C.describe()

In [ ]:
C['Age_intervals']=pd.qcut(C['DAYS_BIRTH_2'], q=4)
C1=C.loc[:,['Predict_Defaut_Avec_CM', 'Age_intervals']]
C2=C1.groupby(['Age_intervals'])['Predict_Defaut_Avec_CM'].sum().reset_index()
C2

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
C2.plot(kind='bar', ax=ax)
ax.set_xlabel('Age', size=15)
ax.set_ylabel('Default rate', size=15)
ax.tick_params(axis='x', labelrotation =90)

In [ ]:
#On voit que ce sont les jeunes ce qui en gros on tendance à être le plus en défaut.


#C.4.2 Features Importance Locale (c'est à dire par client -> LIME):

In [ ]:
##ATTENTION: On n'a pas utilisé SHAP car il a un bug, bug que j'ai signaler sur le github concerné.

#X_train_req.shape

In [ ]:
#explainer = shap.Explainer(lgb_best, X_train_req, check_additivity=False)
#shap_values = explainer(X_train_req)

In [ ]:
#Formulation générale de LIME:

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(training_data= X_train_req.values, feature_names=X_train_req.columns, class_names=['Not Default','Default'], mode='classification')

In [ ]:
#Sauvegardons le "explainer" de "LIME" comme un pickle string.
#Cette fois-ci en utilisant "dill", car "pickel" ne fonctionne pas avec "LIME"

#create an iterator object with write permission - expl.pkl

In [ ]:
with open('expl_pkl', 'wb') as files:
    dill.dump(explainer, files)

In [ ]:
#Vérification, en réouvrant le "explainer" en format "pickel".

In [ ]:
#with open('expl_pkl', 'rb') as files:
#    explainer_pickle = dill.load(files)

In [ ]:
#def explain(data_row, predict_fn):
#    np.random.seed(16)
#    return explainer_pickle.explain_instance(data_row, predict_fn,num_features=20)

In [ ]:
#a. Résultat/Interprétation/Explication du modèle pour la 2ème ligne du dataset:



#ATTENTION:
#Les explications des 'features importances' avec LIME sont le résultat d'un processus d'échantillonnage aléatoire, 
#on ne dois donc pas obtenir exactement la même explication à chaque fois, à moins qu'on définisse le 'random seed' 
#avant chaque exécution de l''explain_instance'.

#Faisons une fonction qui le fasse:

In [ ]:
def explain(data_row, predict_fn):
    np.random.seed(16)
    return explainer.explain_instance(data_row, predict_fn,num_features=20)

In [ ]:
#Faisons plusieurs test:

In [ ]:
exp=explain(X_test_req.iloc[1],lgb_best.predict_proba)

exp.show_in_notebook(show_table=True)

In [ ]:
exp=explain(X_test_req.iloc[1],lgb_best.predict_proba)

exp.show_in_notebook(show_table=True)

In [ ]:
#On a bien le même résultat.

#On voit que pour la 2ème ligne, on a:
# -un client "En Défaut";
# -pour lequel les "valeur en orange" du dernier tableau, et/ou les valeurs de la colonne "Not Default" du tableau du
#milieu sont les raisons pour lesquelles le client est considéré comme "Défaut".


#b. Résultat/Interprétation/Explication du modèle pour la 4ème ligne du dataset:

In [ ]:
exp=explain(X_test_req.iloc[3],lgb_best.predict_proba)

exp.show_in_notebook(show_table=True, show_all=False)

In [ ]:
#Cette fois-ci, le client de la 10ème ligne est considéré comme "Sain" compte tenu des features en bleu.


#c. Résultat/Interprétation/Explication du modèle pour le client au 'SK_ID_CURR'=150932.

In [ ]:
r=X_test_req[X_test_req['SK_ID_CURR']==341512].index[0]
r

In [ ]:
exp=explain(X_test_req.iloc[r],lgb_best.predict_proba)

exp.show_in_notebook(show_table=True)

In [ ]:
#d. Cas général: faisant appel à introduire le 'SK_ID_CURR' du client.

In [ ]:
n = float(input('What is your SK_ID_CURR?'))
s=X_test_req[X_test_req['SK_ID_CURR']==n].index[0]

exp_s=explain(X_test_req.iloc[s],lgb_best.predict_proba)

exp_s.show_in_notebook(show_table=True)

In [ ]:
#Maintenant dans le 'test set'

In [ ]:
n = float(input('What is your SK_ID_CURR?'))
s=test_df_n[test_df_n['SK_ID_CURR']==n].index[0]

exp_s=explain(test_df_n.iloc[s],lgb_best.predict_proba)

exp_s.show_in_notebook(show_table=True)

In [ ]:
#----------------------------------------------------------------------------------------------------------------------